In [ ]:
#Import libraries
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import squarify as sq
from matplotlib.ticker import PercentFormatter
from fim import arules
from operator import attrgetter
import matplotlib.colors as mcolors
warnings.filterwarnings("ignore")
import matplotlib.ticker as tkr
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
df = pd.read_csv('data extract for customer segmentation efood market.csv')

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum() * 100 / len(df)

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.duplicated().sum()  

# Cleaning Steps

In [ ]:
#Remove the missing observations from the dataset

df = df.dropna(subset=['items_sold'])

df.dropna(inplace=True)



In [ ]:
#df['total_sales'] = df['product_price'] * df['items_sold']

df['order_timestamp'] = pd.to_datetime(df['order_timestamp'])

df['registered_at'] = pd.to_datetime(df['registered_at'])

df['items_sold'] = df['items_sold'].astype(int)

# EDA

In [ ]:
#How many unique items in the dataset?

unique_items = df['product_name'].nunique()

print("Number of unique items:", unique_items)

In [ ]:
#How many products in the dataset?

df["product_name"].value_counts()

In [ ]:
#Examine number of transactions per customer 

customer_counts = df["customer_id"].value_counts().sort_values(ascending=False).reset_index()

customer_counts.columns = ['Customer_ID', 'Count']

customer_counts

In [ ]:
# Examine and Plot top25 customers

customer_counts_top25 = customer_counts.head(25)

fig, ax = plt.subplots(figsize = (10, 5))

sns.barplot(data=customer_counts_top25,x = customer_counts_top25["Count"],y = customer_counts_top25["Customer_ID"],
            order = customer_counts_top25["Customer_ID"],  orient = 'h', palette = "Reds_r")

plt.title("Customers that have most transactions")
plt.ylabel("Customers")
plt.xlabel("Transaction Count")

plt.show()

In [ ]:
#Examine Total Sales per Product

product_sales = df.groupby('product_name')['order_value'].sum()

print(product_sales)

In [ ]:
product_sales_sorted = product_sales.sort_values(ascending=False)

print(product_sales_sorted)

In [ ]:
# Examine and Plot top10 Products according to Total Sales



product_sales = df.groupby('product_name')['order_value'].sum()

sorted_product_sales = product_sales.sort_values(ascending=False)

top_10_products = sorted_product_sales.head(10)


fig = go.Figure(data=go.Bar(
    x=top_10_products.index,
    y=top_10_products.values,
    text=top_10_products.values,
    textposition='auto'
))

fig.update_layout(
    title='Top 10 Products by Total Sales',
    xaxis_title='Product',
    yaxis_title='Total Sales',
    xaxis_tickangle=-45,
)

fig.show()

# Market Basket Analysis / Association Rules

In [ ]:
# inputs
supp = 2 # minimum support of an assoc. rule (default: 10)
conf = 20 # minimum confidence of an assoc. rule (default: 80%)
report = 'aSCl'

In [ ]:
report_colnames = {
    'a': 'Support',
    's': 'support_itemset_relative',
    'S': 'support_itemset_relative_pct',
    'b': 'support_bodyset_absolute',
    'x': 'support_bodyset_relative',
    'X': 'support_bodyset_relative_pct',
    'h': 'support_headitem_absolute',
    'y': 'support_headitem_relative',
    'Y': 'support_headitem_relative_pct',
    'c': 'confidence',
    'C': 'Confidence',
    'l': 'Lift',
    'L': 'lift_pct',
    'e': 'evaluation',
    'E': 'evaluation_pct',
    'Q': 'support of the empty set (total number of transactions)'
    }

In [ ]:
# display docs
#??arules

In [ ]:
??arules

In [ ]:
Customers_Products=df.groupby('customer_id')['product_name'].apply(list)
Customers_Products

In [ ]:
# Create list only with Customers and Products
Customers_Products=df.groupby('customer_id')['product_name'].apply(list)


# Create list of lists to pass it into PyFim

Customers_Products_List=Customers_Products.to_list()
Customers_Products_List

In [ ]:
# Run apriori algorithm to creeate associations
result = arules(Customers_Products_List, supp=supp, conf=conf, report=report)

#Uncomment and run in case you want to see parameters of association rules
##??arules


#Creat dataframe wiht columns based on the report variables and sorted based on the "Associations_Sorted_based_on"
colnames = ['Cons_Product', 'Product'] + [report_colnames.get(k, k) for k in list(report)]
df_rules = pd.DataFrame(result, columns=colnames)
df_rules = df_rules.sort_values(report_colnames["a"], ascending=False)

#Change order of columns antecedent and consequent
df_rules = df_rules[['Product', 'Cons_Product']+ [report_colnames.get(k, k) for k in list(report)]]


#Print the numbers of rules created
print(df_rules.shape)

#Print top 10 and last 10 rules (the last 10 will be shown)
df_rules.head(10)

In [ ]:
df_rules.to_csv('Support.csv')

In [ ]:
df_rules[df_rules['Lift'] < 2]

In [ ]:
df_rules.sort_values(by=['Confidence'], ascending=False).head(10)

In [ ]:
df_rules.sort_values(by=['Support'], ascending=False).head(5)

In [ ]:
df_rules.sort_values(by=['Lift'], ascending=False).head(10)

# What percentage of Orders do the top 10 users of each city contribute to their city?

In [ ]:
# Filter cities with more than 1000 orders
filtered_cities = df.groupby('geographical_region_of_user').filter(lambda x: x['order_id'].nunique() > 1000)

# Group by city and user, count the orders and calculate the sum of their amounts
city_user_orders = filtered_cities.groupby(['geographical_region_of_user', 'customer_id']).agg({'order_id': 'nunique', 'purchase_price': 'sum'})

# Reset index for easier manipulation
city_user_orders = city_user_orders.reset_index()

# Sort by city and order count, and keep only the top 10 users per city
top_users = city_user_orders.groupby('geographical_region_of_user').apply(lambda x: x.nlargest(10, 'order_id')).reset_index(drop=True)

# Calculate the total orders for each city
city_total_orders = filtered_cities.groupby('geographical_region_of_user')['order_id'].nunique().reset_index(name='total_orders')

# Merge the top_users DataFrame with city_total_orders
top_users_with_total = pd.merge(top_users, city_total_orders, on='geographical_region_of_user')

# Calculate the percentage of each user's orders to the total orders of their city
top_users_with_total['percentage'] = (top_users_with_total['order_id'] / top_users_with_total['total_orders']) * 100

city_percentage = top_users_with_total.groupby('geographical_region_of_user')['percentage'].sum().reset_index()

# Sort the DataFrame by the sum of percentages in descending order
city_percentage = city_percentage.sort_values('percentage', ascending=False)

# Reset the index starting from 1
city_percentage = city_percentage.reset_index(drop=True)
city_percentage.index += 1

# Display the result
city_percentage

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(city_percentage['geographical_region_of_user'], city_percentage['percentage'])
plt.xticks(rotation=45)
plt.xlabel('Πόλεις')
plt.ylabel('% Συμμετοχής')
plt.title('% Συμμετοχής Τοπ Χρηστών ανά Πόλη')

plt.show()
None

# Cohort Analysis

In [ ]:
 #Create the cohort and order_month variables
df['order_month'] = df['order_timestamp'].dt.to_period('M')

df['cohort']= df.groupby('customer_id')['order_timestamp'].transform('min').dt.to_period('M')

df

In [ ]:
#We aggregate the data per cohort and order_month and count the number of unique customers in each group.

df_cohort = df.groupby(['cohort', 'order_month']).agg(n_customers=('customer_id',
                                                                   'nunique')).reset_index(drop=False)

In [ ]:
df_cohort['period_number'] = (df_cohort.order_month - df_cohort.cohort).apply(attrgetter('n'))

df_cohort


In [ ]:
# We aggregate the data per cohort and order_month and count the number of unique customers in each group.

cohort_pivot = df_cohort.pivot_table(index='cohort', 
                                     columns='period_number', 
                                     values='n_customers')
cohort_pivot

In [ ]:
cohort_size = cohort_pivot.iloc[:, 0]
retention_matrix = cohort_pivot.divide(cohort_size, axis=0)

In [ ]:
with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(12, 8), sharey=True, gridspec_kw={'width_ratios': [1, 11]})
    
    # retention matrix
    sns.heatmap(retention_matrix, 
                mask=retention_matrix.isnull(), 
                annot=True, 
                fmt='.0%', 
                cmap='RdYlGn', 
                ax=ax[1])
    ax[1].set_title('Monthly Cohorts: User Retention', fontsize=16)
    ax[1].set(xlabel='# of periods',
              ylabel='')

    # cohort size
    cohort_size_df = pd.DataFrame(cohort_size).rename(columns={0: 'cohort_size'})
    white_cmap = mcolors.ListedColormap(['grey'])
    sns.heatmap(cohort_size_df, 
                annot=True, 
                cbar=False, 
                fmt='g', 
                cmap=white_cmap, 
                ax=ax[0])

    fig.tight_layout()

# RFM_Pareto

In [ ]:
df['Total_Sales'] = df['product_price'] * df['items_sold']

In [ ]:
df['order_timestamp'].max()

In [ ]:
df['order_timestamp'].min()

In [ ]:
# Find Recency. Take as reference day the maximum date
df["Recency"] = (df["order_timestamp"].max()- df["order_timestamp"]).dt.days
df.head()

In [ ]:
df['Recency'].max()

In [ ]:
# For each customer find:

# Recency : minimum Recency

# Frequency: count unique number of days of purchases

# Monetary: summary of Total Sales

df_rfm = df.groupby(["customer_id"]).agg({"Recency":np.min,
                                           "order_timestamp":pd.Series.nunique,
                                           "Total_Sales":np.sum}).reset_index()
df_rfm

In [ ]:
df_rfm.rename(columns={"order_timestamp":"Frequency","Total_Sales":"Monetary"},inplace=True)
df_rfm

In [ ]:
# Create Scores RFM scores based on quantiles of distribution


#Date from customer's last purchase.The nearest date gets 4 and the furthest date gets 1.
df_rfm["recency_score"] = pd.qcut(df_rfm["Recency"].rank(method="first"),4,labels=[4,3,2,1])


# Total number of purchases.The least frequency gets 1 and the maximum frequency gets 4.
df_rfm["frequency_score"] = pd.qcut(df_rfm["Frequency"].rank(method="first"),4,labels=[1,2,3,4])


#Total spend by the customer.The least money gets 1, the most money gets 4.
df_rfm["monetary_score"] = pd.qcut(df_rfm["Monetary"].rank(method="first"),4,labels=[1,2,3,4])

In [ ]:
df_rfm["RFM_Segment"] = df_rfm["recency_score"].astype(str) + df_rfm["frequency_score"].astype(str) + df_rfm["monetary\
_score"].astype(str)
df_rfm

In [ ]:
df_rfm["RFM Score"] = df_rfm[["recency_score","frequency_score","monetary_score"]].sum(axis=1)
df_rfm

In [ ]:
# Use regex and dictionaries values to categorize numbers from customers

segt_map = {
    r'[3-4][3-4]4': 'VIP',
    r'[2-3-4][1-2-3-4]4': 'Top Recent',
    r'1[1-2-3-4]4': 'Top at Risk ',

    
    
    r'[3-4][3-4]3': 'High Promising',
    r'[2-3-4][1-2]3': 'High New',
    r'2[3-4]3': 'High Loyal',

    
    
    r'[3-4][3-4]2': 'Medium Potential',
    r'[2-3-4][1-2]2': 'Medium New',
    r'2[3-4]2': 'Medium Loyal',

    
    
    r'4[1-2-3-4]1': 'Low New',
    r'[2-3][1-2-3-4]1': 'Low Loyal',
    
    r'1[1-2-3-4][1-2-3]': 'Need Activation'
}

In [ ]:
df_rfm["Segment_labels"] = df_rfm["RFM_Segment"]
df_rfm["Segment_labels"] = df_rfm["Segment_labels"].replace(segt_map, regex=True)
df_rfm.head()

In [ ]:
df_rfm['customer_id'].nunique()

# Calculate Customers per Segments and all Stats per Segment

In [ ]:
segements_grouped = df_rfm[["Segment_labels", "Recency","Frequency","Monetary"]].\
groupby("Segment_labels").agg(["mean","count","max"]).round().reset_index()


segements_grouped = df_rfm.groupby(["Segment_labels"]).agg({'Monetary': np.sum,
                                   "customer_id": pd.Series.nunique,
                                    "Recency": np.mean,                     
                                     "Frequency": np.mean,
                                                        }).reset_index()

segements_grouped["Monetary%"] = segements_grouped["Monetary"]/segements_grouped["Monetary"].sum()*100
segements_grouped["Count%"] = segements_grouped["customer_id"]/segements_grouped["customer_id"].sum()*100

segements_grouped.rename(columns={'Monetary':'TotalMonetary','customer_id':'TotalCustomers',
                                 'Recency':'MeanRecency','Frequency':'MeanFrequency',
                                 "Count%":"Customers%"},inplace= True)


segements_grouped = segements_grouped.sort_values(by=['Monetary%'], ascending=False)
segements_grouped

In [ ]:
segements_grouped[['Segment_labels', 'TotalMonetary', 'TotalCustomers']]

In [ ]:
segements_grouped.info()

In [ ]:
segements_grouped['TotalMonetary'] = segements_grouped['TotalMonetary'] * 1000
segements_grouped

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

sns.barplot(
    data=segements_grouped,
    x=segements_grouped["TotalMonetary"],
    y=segements_grouped["Segment_labels"],
    #rder=segements_grouped["Segment_labels"],
    orient='h',
    palette="crest"
)

#ax.bar_label(ax.containers[0], label_type='edge')  # Adjust label_type parameter
plt.title(" € ανά Κατηγορία")
plt.ylabel("Κατηγορία")
plt.xlabel("")


x_max=segements_grouped["TotalMonetary"].max()
#lt.xlim(0, x_max * 1.1)

#lt.tight_layout()

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

sns.barplot(
    data=segements_grouped,
    x=segements_grouped["TotalCustomers"],
    y=segements_grouped["Segment_labels"],
    order=segements_grouped["Segment_labels"],
    orient='h',
    palette="crest"
)

#ax.bar_label(ax.containers[0], label_type='edge')  # Adjust label_type parameter
plt.title("Πλήθος Χρηστών")
plt.ylabel("Κατηγορία")
plt.xlabel("")


x_max = segements_grouped["TotalCustomers"].max()
plt.xlim(0, x_max * 1.1)

plt.tight_layout()

plt.show()

In [ ]:
segements_grouped.to_csv(index=False)

In [ ]:
from pathlib import Path  
filepath = Path('Maria Vasilaki/PYTHON/BigBlueDataAcademy/BOOTCAMP/Project/out.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
segements_grouped.to_csv(filepath)

In [ ]:
# Plot Segments in a Treemap

fig = plt.gcf()
ax = fig.add_subplot()
fig.set_size_inches(13, 8)
sq.plot(sizes=segements_grouped["Customers%"], 
                      label=['VIP', 
                            "Top Recent", 
                            'High Promising', 
                            'Need Activation', 
                            'Top at Risk', 
                            'High New', 
                            'High Loyal',
                            'Medium New',
                            'Medium Potential',
                            'Low Loyal',
                            'Medium Loyal',
                            'Low New'], 
                            alpha=0.8, 
                            color=["#282828", "#04621B", "#971194", "#F1480F",  "#4C00FF", 
           "#FF007B", "#9736FF", "#8992F3", "#B29800", "#80004C", "blue", "cyan"])#"#A4E919"
plt.title("ALL", fontsize=18, fontweight="bold")
plt.axis('off')
plt.show()

In [ ]:
# Pie Chart: Percentage of Customers in Each Segment
plt.figure(figsize=(8, 8))
plt.pie(segements_grouped['Customers%'], labels=segements_grouped['Segment_labels'], autopct='%1.1f%%')
plt.title('Customer Segmentation - Percentage of Customers in Each Segment')
plt.axis('equal')
plt.show()

In [ ]:
# Stacked Bar Chart: Percentage of Customers in Each Segment
plt.figure(figsize=(10, 6))
segments = segements_grouped['Segment_labels']
customers_percentage = segements_grouped['Customers%']

# Define colors for each segment
colors = ['lightblue', 'lightgreen', 'orange', 'pink']

# Create the stacked bar chart
plt.barh(segments, customers_percentage, color=colors)
plt.xlabel('Percentage of Customers')
plt.ylabel('Segment')
plt.title('Customer Segmentation - Percentage of Customers in Each Segment')

plt.show()

In [ ]:
#Here we'll try to analyze what % of customers produce what % of total sales. 
#In this way we'll be able to identify our most valuable customers.

segements_grouped

##  If we observe our results we will notice that VIP and Top Recent Customers (21% of customers) produce 68% of Monetary Value (Sales), Mention that we examined the pareto that expects 80% and find close to 70, here is the relevant diagram.¶
We could apply our graph at first on our created segments and then on individual customers¶

In [ ]:
segments_pareto = segements_grouped[["Segment_labels","TotalMonetary"]]
segments_pareto["CumulativePercentage"] = (segments_pareto["TotalMonetary"].cumsum()/ 
                                      segments_pareto["TotalMonetary"].sum()*100).round(2)

In [ ]:
#define aesthetics for plot
color1 = 'steelblue'
color2 = 'red'
#line_size = 1

#create basic bar plot
fig, ax = plt.subplots()
ax.bar(segments_pareto['Segment_labels'], segments_pareto['TotalMonetary'], color=color1)


#add cumulative percentage line to plot
ax2 = ax.twinx()
ax2.plot(segments_pareto['Segment_labels'], segments_pareto['CumulativePercentage'], color=color2, marker="D", ms=1)
ax2.yaxis.set_major_formatter(PercentFormatter())
ax2.axhline(80, color = "#008878", linestyle = "dashed", alpha = 1 )


    
#specify axis colors
ax.tick_params(axis='y', colors=color1, labelsize= 8)
#ax.set_xticklabels([])
ax2.tick_params(axis='y', colors=color2, labelsize= 8)

ax.tick_params(axis='x', labelsize= 6)
ax.yaxis.set_major_formatter(tkr.FuncFormatter(lambda y,  p: format(int(y), ',')))

#ax.tick_params(axis='x', labelsize= )
#display Pareto chart
plt.show()

In [ ]:
df_grouped = df.groupby("customer_id").\
    agg({"Total_Sales": np.sum
        })

df_grouped = df_grouped.sort_values("Total_Sales", ascending = False).reset_index()
df_grouped

In [ ]:
df_grouped["CumulativePercentage"] = (df_grouped["Total_Sales"].cumsum()/ 
                                      df_grouped["Total_Sales"].sum()*100).round(2)
                                      

df_grouped

In [ ]:
 # We focus on the 80.00% part of sales to find what number of customers it is generated from.

number_of_customers = df_grouped[(df_grouped["CumulativePercentage"] == 80.00)]
number_of_customers 

In [ ]:
13092/len(df_grouped)

In [ ]:
13086/len(df_grouped)

## So we see that each of them represent 31% of total customers.
In this way we just found out that the highest 31% of customers produce 80% of total Sales.
So the Pareto Principle roughly applies to our example. Now let's work in order to porduce also the according plot which will also visualize it.

In [ ]:
#define aesthetics for plot
#color1 = 'steelblue'
#color2 = 'red'
#line_size = 1

#create basic bar plot
#fig, ax = plt.subplots()
#ax.bar(df_grouped['customer_id'], df_grouped['Total_Sales'], color=color1)


#add cumulative percentage line to plot
#ax2 = ax.twinx()
#ax2.plot(df_grouped['customer_id'], df_grouped['CumulativePercentage'], color=color2, marker="D", ms=1)
#ax2.yaxis.set_major_formatter(PercentFormatter())
#ax2.axhline(80, color = "#008878", linestyle = "dashed", alpha = 1 )


    
#specify axis colors
#ax.tick_params(axis='y', colors=color1, labelsize= 8)
#ax.set_xticklabels([])

#ax2.tick_params(axis='y', colors=color2, labelsize= 8)

#ax.tick_params(axis='x', labelsize= 1)

#ax.tick_params(axis='x', labelsize= )
#display Pareto chart
#plt.show()

# Patterns

In [ ]:
df['order_month'] = df['order_timestamp'].dt.to_period('M')

metrics_df = df.groupby('order_month').agg(total_sales=('order_value', 'sum'),
                                           num_orders=('order_id', 'count'),
                                           avg_order_value=('order_value', 'mean'))



metrics_df['order_month_str'] = metrics_df.index.astype(str)

fig = px.line(metrics_df, x='order_month_str', y=['total_sales', 'num_orders', 'avg_order_value'],
              labels={'value': 'Metrics', 'order_month_str': 'Order Month'},
              title='Total Sales over Time')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [ ]:
# Calculate total sales by product category


category_metrics = df.groupby('product_category').agg(total_sales=('order_value', 'sum')).reset_index()


top_categories = category_metrics.nlargest(5, 'total_sales')


df_top_categories = df[df['product_category'].isin(top_categories['product_category'])]


df_top_categories['order_month_str'] = df_top_categories['order_month'].astype(str)


category_metrics_top = df_top_categories.groupby(['order_month_str', 'product_category']).agg(total_sales=('order_value', 'sum')).reset_index()


fig = px.line(category_metrics_top, x='order_month_str', y='total_sales', color='product_category',
              labels={'order_month_str': 'Order Month', 'total_sales': 'Total Sales'},
              title='Purchase Patterns by Top 10 Product Categories')

fig.show()


In [ ]:
total_unique_orders = df['order_id'].nunique()
print("Total Unique Orders:", total_unique_orders)